Esse notebook tem como objetivo gerar varios DataFrames baseando-se nas colunas do dataframe 'pgm-dataset-processado.csv'

As colunas de onde virão os Dataframes:
* Assuntos (Assuntos do CNJ)
* Orgão Julgador
* Documentos (Contagem)
* Classe Processo

# 0. Imports

In [1]:
import os
import pandas as pd
from typing import List

from sklearn.feature_extraction.text import CountVectorizer

# 0.1 - Funções

In [26]:
def gera_df(list_: List[List[str]], most_common=None):
    '''
    Converte uma lista de listas em um DataFrame.

    Cada sublista da lista de entrada é transformada em uma linha do DataFrame,
    onde cada coluna representa um valor único presente em alguma sublista.
    Se o valor estiver presente na sublista, a célula recebe 1; caso contrário, 0.
    '''
    if most_common:
        unique_values = most_common
    else:    
        unique_values = sorted(set(item for sublist in list_ for item in sublist))

    df = pd.DataFrame([{val: sublist.count(val) if val in sublist else 0 for val in unique_values} for sublist in list_])

    return df

def balance_df(df: pd.DataFrame, balance_on:str = 'setorDestino', n_samples:int = 500):
    unique_classes = df[balance_on].unique()
    
    dfs_list = []

    for classe in unique_classes:
        if len(df[df[balance_on] == classe]) >= n_samples:
            dfs_list.append(df[df[balance_on] == classe].sample(n=n_samples, replace=False))
        else:
            dfs_list.append(df[df[balance_on] == classe].sample(n=n_samples, replace=True))
    
    return pd.concat(dfs_list)

# 1. Data Loading

In [3]:
df = pd.read_csv('/home/esdras-daniel/Documentos/Python/PGM-DataAnalysis/data/processed/pgm-dataset-processado.csv')

path_to_be_saved = '/home/esdras-daniel/Documentos/Python/PGM-DataAnalysis/data/processed'

In [36]:
df['setorDestino'].value_counts()

setorDestino
Fiscal            6686
Administrativa    5403
Contabilidade     1338
Judicial          1137
Saúde              443
Meio Ambiente      155
Patrimonial        121
Name: count, dtype: int64

# 2. Criação de Dataframes

## 2.1 - Assuntos

In [4]:
assuntos = df['assuntos'].apply(lambda x: x.split(';'))

In [5]:
df_assuntos = gera_df(assuntos)

In [6]:
# Adiciona o setorDestino
df_assuntos['setorDestino'] = df['setorDestino']

In [7]:
# Salva o DataFrame
df_assuntos.to_csv(os.path.join(path_to_be_saved, 'df_assuntos.csv'), index=False)

## 2.2 - Orgão Julgador

In [8]:
orgaos_julgadores = df['orgaoJulgador'].apply(lambda x: x.split(';'))

In [9]:
df_orgaosJulgadores = gera_df(orgaos_julgadores)

In [10]:
# Adiciona o setorDestino
df_orgaosJulgadores['setorDestino'] = df['setorDestino']

In [11]:
# Salva o DataFrame
df_orgaosJulgadores.to_csv(os.path.join(path_to_be_saved, 'df_orgaosJulgadores.csv'), index=False)

## 2.3 - Documentos

In [12]:
documentos = df['documentos'].apply(lambda x: x.split(';'))

In [13]:
# Recuperando documentos mais frequentes
count_vectorizer = CountVectorizer(tokenizer=lambda x: x.split(';'), max_features=100)
X = count_vectorizer.fit_transform(df['documentos']).todense()

/home/esdras-daniel/Documentos/Python/PGM-DataAnalysis/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [14]:
df_documentos = pd.DataFrame(columns=count_vectorizer.get_feature_names_out(),
                             data=X)

In [15]:
df_documentos['setorDestino'] = df['setorDestino']

In [16]:
# Salva o DataFrame
df_documentos.to_csv(os.path.join(path_to_be_saved, 'df_documentos.csv'), index=False)

## 2.4 - Classe Processo

In [17]:
classes_processo = df['classeProcesso'].apply(lambda x: str(x).split(';'))

In [18]:
df_classes_processo = gera_df(classes_processo)

In [19]:
# Adiciona o setorDestino
df_classes_processo['setorDestino'] = df['setorDestino']

In [20]:
# Salva o DataFrame
df_classes_processo.to_csv(os.path.join(path_to_be_saved, 'df_classe_processo.csv'), index=False)

## 2.5 - DataFrame Balanceado

In [34]:
df_balanceado = balance_df(df, balance_on='setorDestino', n_samples=750)

print(df_balanceado.shape)
print(df_balanceado['setorDestino'].value_counts())

(5250, 8)
setorDestino
Saúde             750
Administrativa    750
Fiscal            750
Contabilidade     750
Judicial          750
Patrimonial       750
Meio Ambiente     750
Name: count, dtype: int64


In [35]:
# Salva o DataFrame
df_balanceado.to_csv(os.path.join(path_to_be_saved, 'pgm-dataset-processado-balanceado.csv'), index=False)